## 로지스틱 회귀로 와인 분류하기
와인의 데이터가 주어졌을 때, 알코올, 도수, 당도, pH 특성을 참고하여 와인 종류를 구분해보자. 와인의 종류는 화이트 와인, 레드 와인 두 종류이며, 화이트와인이 1(양성 클래스)이다

In [4]:
import pandas as pd

# 데이터 불러오기
wine = pd.read_csv('https://bit.ly/wine_csv_data')

# 처음 5개의 샘플 확인
print(wine.head())
print()

# pandas info() 메서드: 데이터프레임의 각 열의 데이터 타입과 데이터 누락 여부 확인
print(wine.info())
print()

# pandas describe() 메서드: 열에 대한 간략한 통계 출력
wine.describe()

   alcohol  sugar    pH  class
0      9.4    1.9  3.51    0.0
1      9.8    2.6  3.20    0.0
2      9.8    2.3  3.26    0.0
3      9.8    1.9  3.16    0.0
4      9.4    1.9  3.51    0.0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB
None



,alcohol,sugar,pH,class
count,6497.000000,6497.000000,6497.000000,6497.000000
mean,10.491801,5.443235,3.218501,0.753886
std,1.192712,4.757804,0.160787,0.430779
min,8.000000,0.600000,2.720000,0.000000
25%,9.500000,1.800000,3.110000,1.000000
50%,10.300000,3.000000,3.210000,1.000000
75%,11.300000,8.100000,3.320000,1.000000
max,14.900000,65.800000,4.010000,1.000000


In [5]:
# 타깃 데이터 떼어내기
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# 훈련셋과 테스트셋 나누기
from sklearn.model_selection import train_test_split

# test_size=0.2: 기본 테스트셋은 25%를 떼내는데,
# test_size를 변경하여 테스트셋을 20%로 조정
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

# 훈련셋과 테스트셋 크기 확인
print(train_input.shape, test_input.shape)

# 알코올, 당도, 산도 스케일이 다르므로 특성 표준화 필요
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

# 모델 훈련
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

# 일단 결과는 언더피팅. 훈련 결과와 테스트 결과 모두 조금 낮은 편

(5197, 3) (1300, 3)
0.7808350971714451
0.7776923076923077
